In [1]:
from __future__ import print_function
from __future__ import division
from keras.utils import plot_model
from keras.models import Sequential, model_from_json
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lib.pretty_testing import predict_test, class_weights_max, confusion_matrix
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

Using TensorFlow backend.


In [2]:
top_words = 5000
embedding_vector_length = 32
maxlen = 100
epochs = 25
#classes = ['poteri', 'scadenza', 'assemblea', 'clausola', 'non_riconducibile']
classes = ['poteri', 'assemblea', 'clausola', 'non_riconducibile']
train_also_test = True

In [3]:
# load prepared data
with open('../extraction/dataset.p') as f:
    data, labels = pickle.load(f)

In [4]:
padded_data = sequence.pad_sequences(data, maxlen, padding="pre", truncating="post", value=0, dtype='uint32')
labels = np.array(labels)

In [5]:
def build_model():
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=maxlen, mask_zero=True))
    model.add(LSTM(32, dropout=0.2)) #return_sequences=True
    model.add(Dense(len(classes), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model


In [ ]:
#plot_model(model, to_file='model.png')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, train_size=0.8, stratify=labels)
y_train_cat = to_categorical(y_train, num_classes=len(classes))
y_test_cat = to_categorical(y_test, num_classes=len(classes))

In [ ]:
[l.get_weights()[0].shape for l in model.layers]

In [7]:
class_weights = class_weights_max(labels)

In [ ]:
class_weights

In [ ]:
model = build_model()
model.summary()
h = model.fit(X_train,  y_train_cat, epochs=epochs, batch_size=32, verbose=0, class_weight=class_weights)
#h.history

In [ ]:
df = pd.DataFrame(h.history)
p = df.plot(figsize=(16,8))

In [ ]:
y_pred = predict_test(model, X_test, y_test, classes)

In [8]:
if train_also_test:
    model = build_model()
    labels_cat = to_categorical(labels)
    model.fit(padded_data,  labels_cat, epochs=epochs, batch_size=32, verbose=0, class_weight=class_weights)
    with open('models/extraction_model_30_all.json','w') as f:
        f.write(model.to_json())
    model.save_weights("models/extraction_weights_30_all.h5")

In [9]:
labels_cat.shape

(5566, 4)

In [ ]:
precision_recall_fscore_support(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
with open('models/extraction_model_30.json','w') as f:
    f.write(model.to_json())

In [ ]:
model.save_weights("models/extraction_weights_30.h5")

In [ ]:
precision_recall_fscore_support(y_test, y_post)